# 해당 파일(stage.ipynb)의 목적

지난 과정 (50lv_maker.ipynb)에서 50 레벨을 기준으로한 능력치를 산출했습니다.
이를 토대로 포켓몬 배틀을 시행해 확률을 포켓몬 배틀에서 이길 확률을 구하고자 합니다.

## 1. 포켓몬 배틀의 전략 가정

기술의 효과를 무시하고, 기술의 위력은 모두 50으로 가정하였습니다. 따라서 기술에 영향을 미치는 것은 저의 공격력(특수공격력)과 상대방의 방어력(특수방어력), 그리고 상대 포켓몬의 타입에 따른 데미지 여부(예: 비행 타입 포켓몬은 전기 기술에 두 배의 데미지) 입니다.

*** 실제 포켓몬 게임에서는 포켓몬 본인의 타입 외에 다른 타입의 기술도 사용 가능하지만 (예: 고우스트(고스트 타입)의 사이코키네시스(에스퍼 타입)) 분석을 진행하기 위해 해당 타입의 포켓몬은 본인이 해당한 타입과 노말 타입의 기술만 사용 가능하다고 가정합니다. 

포켓몬이 사용할 수 있는 기술, 즉 (본인이 해당한 타입과 노말) * (물리공격, 특수공격) 중 상대 포켓몬에게 가장 큰 데미지를 입히는 기술을 사용한다고 가정합니다.

## 2. 포켓몬 데미지 공식

참고 : http://tip.daum.net/question/58394183

In [1]:
# 분석에 필요한 패키지를 불러옵니다.

import pandas as pd
import numpy as np
import random as r

In [2]:
# 포켓몬에 대한 정보를 담을 Pokemon 클래스를 생성합니다.

class Pokemon:
    def __init__(self, info: pd.Series):
        self.info = info
        self.hp = info['50lv_hp']
        self.died = False
    
    def getHurt(self, damage):
        self.hp -= damage
        
# getDied는 포켓몬 배틀에서 포켓몬이 죽었는지 확인합니다.
    def getDied(self):
        if self.hp <= 0 :
            self.died = True
        return self.died
    
    def getAttack(self):
        return self.info['50lv_attack']

    def getDefense(self):
        return self.info['50lv_defense']

    def getSpAttack(self):
        return self.info['50lv_sp_attack']

    def getSpDefense(self):
        return self.info['50lv_sp_defense']

    def getSpeed(self):
        return self.info['50lv_speed']
    
    def getType(self):
        if pd.isnull(self.info['type2']):
            return (self.info['type1'], )
        else:
            return (self.info['type1'], self.info['type2'], )
        
    def getAgainst(self, s):
        return self.info['against_' + fightingChange(s)]


# 포켓몬 배틀 시에 사용될 Battle 클래스를 생성합니다.

class Battle:
    rdnum = range(85,101)
    rdpdf = np.array([0.769, 0.513, 0.769, 0.513, 0.769, 0.769, 0.513, 0.769, 0.513, 0.769, 0.513, 0.769, 0.513, 0.769, 0.513, 0.256],
                    dtype = np.float32)/ np.array([0.769, 0.513, 0.769, 0.513, 0.769, 0.769, 0.513, 0.769, 0.513, 0.769, 0.513, 0.769,
                    0.513, 0.769, 0.513, 0.256], dtype = np.float32).sum()
    
    def __init__(self, p1: Pokemon, p2: Pokemon):
        self.p1 = p1
        self.p2 = p2
        self.p1_damage = 0
        self.p2_damage = 0
        self.priority = 0
        
    def initialize(self):
        self.p1.hp = self.p1.info['50lv_hp']
        self.p2.hp = self.p2.info['50lv_hp']
        self.p1.died = False
        self.p2.died = False
        self.p1_damage = self.getDamage(self.p1, self.p2)
        self.p2_damage = self.getDamage(self.p1, self.p2)
        self.priority = self.getPriority()
        
    def check(self):
        if self.p1_damage == 0 and self.p2_damage == 0:
            return 0
        elif self.p1_damage > 0 and self.p2_damage == 0:
            return 1
        elif self.p1_damage == 0 and self.p2_damage > 0:
            return 2
        else:
            return 4


# start() 는 첫 번째 포켓몬이 이기면 1, 두 번째 포켓몬이 이기면 2, 두 포켓몬 다 상대에게 입힐 수 있는 데미지가 0이어서
# 포켓몬 배틀이 끝나지 않을 시 0을 반환합니다.

    def start(self):
        s = self.check()
        if s < 4:
            return s
        while True:
            p1_d = self.p1_damage * np.random.choice(a = self.rdnum, size = 1, replace = False, p = self.rdpdf) / 100.0
            p2_d = self.p2_damage * np.random.choice(a = self.rdnum, size = 1, replace = False, p = self.rdpdf) / 100.0
            
            if self.priority == 1:
                self.p2.getHurt(p1_d)
                if self.p2.getDied():
                    return 1
                self.p1.getHurt(p2_d)
                if self.p1.getDied():
                    return 2
            
            elif self.priority == 2:
                self.p1.getHurt(p2_d)
                if self.p1.getDied():
                    return 2
                self.p2.getHurt(p1_d)
                if self.p2.getDied():
                    return 1
            
            else:
                turn = r.randint(0,1)
                
                if turn == 0:
                    self.p2.getHurt(p1_d)
                    if self.p2.getDied():
                        return 1
                    self.p1.getHurt(p2_d)
                    if self.p1.getDied():
                        return 2
                
                if turn == 1:
                    self.p1.getHurt(p2_d)
                    if self.p1.getDied():
                        return 2
                    self.p2.getHurt(p1_d)
                    if self.p2.getDied():
                        return 1
                
            
# getDamage(attacker, defender)는 주어진 가정 하에 attacker가 defender에게 입힐 수 있는 최고 데미지를 구합니다.
            
    def getDamage(self, attacker: Pokemon, defender: Pokemon):
        if ('normal') == attacker.getType():
            nor_ph_damage = (22*attacker.getAttack()/defender.getDefense() + 2) * nor_correction * defender.getAgainst('normal')
            nor_sp_damage = (22*attacker.getSpAttack()/defender.getSpDefense() + 2) * nor_correction * defender.getAgainst('normal')
            return max(nor_ph_damage, nor_sp_damage)
        
        else:
            nor_correction = self.getNorCorrection(attacker)
            type_correction = self.getTypeCorrection(attacker, defender)
            nor_ph_damage = (22*attacker.getAttack()/defender.getDefense() + 2) * nor_correction * defender.getAgainst('normal')
            nor_sp_damage = (22*attacker.getSpAttack()/defender.getSpDefense() + 2) * nor_correction * defender.getAgainst('normal')
            tp_ph_damage = (33*attacker.getAttack()/defender.getDefense() + 2) * type_correction
            tp_sp_damage = (33*attacker.getAttack()/defender.getDefense() + 2) * type_correction
            return max(nor_ph_damage, nor_sp_damage, tp_ph_damage, tp_sp_damage)
    
    def getNorCorrection(self, p: Pokemon):
        for i in p.getType():
            if i == 'normal':
                return 1.5
        return 1
        
    def getTypeCorrection(self, attacker: Pokemon, defender: Pokemon):
        r = 0
        for i in attacker.getType():
            s = defender.getAgainst(i)
            if r < s:
                r = s
        return r
    
    
# getPriority() 는 포켓몬 간의 스피드를 비교해 공격의 우선 순위를 결정합니다.
# 만약 두 포켓몬의 스피드가 같을 시 매 턴마다 랜덤으로 선공과 후공이 결정됩니다.

    def getPriority(self):
        s1 = self.p1.getSpeed()
        s2 = self.p2.getSpeed()
        if s1 > s2:
            return 1
        elif s1 < s2:
            return 2
        else:
            return 0

def fightingChange(s):
    if s == 'fighting':
        s = s[:-3]
    return s

In [3]:
# 50 레벨을 기준으로 한 포켓몬 정보를 불러옵니다.

pokemon = pd.read_csv("pokemon_stat_50lv.csv", encoding = "utf-8")

In [4]:
# 포켓몬 간 대결을 시킵니다.

n = len(pokemon)
result = np.zeros((n, n))

for i in range(n):
    pokemon1 = Pokemon(pokemon.loc[i,:])
    for j in range(i+1):
        rate = 0
        pokemon2 = Pokemon(pokemon.loc[j,:])
        battle = Battle(pokemon1, pokemon2)
        
        # 같은 상대방끼리 포켓몬 배틀을 100회 진행해 이기는 비율을 구합니다.
        
        for k in range(100):
            battle.initialize()
            winner = battle.start()
            if winner == 1:
                rate += 1
            elif winner == 0:
                rate += -1
                break
        rate = rate/100.0
        
        if rate >= 0:
            if i != j:
                result[i,j] = rate
                result[j,i] = 1.0 - rate
            else:
                result[i,j] = rate

        else:
            result[i,j] = -1.0
            result[j,i] = -1.0
    
    print(i, end = "/ ")

0/ 1/ 2/ 3/ 4/ 5/ 6/ 7/ 8/ 9/ 10/ 11/ 12/ 13/ 14/ 15/ 16/ 17/ 18/ 19/ 20/ 21/ 22/ 23/ 24/ 25/ 26/ 27/ 28/ 29/ 30/ 31/ 32/ 33/ 34/ 35/ 36/ 37/ 38/ 39/ 40/ 41/ 42/ 43/ 44/ 45/ 46/ 47/ 48/ 49/ 50/ 51/ 52/ 53/ 54/ 55/ 56/ 57/ 58/ 59/ 60/ 61/ 62/ 63/ 64/ 65/ 66/ 67/ 68/ 69/ 70/ 71/ 72/ 73/ 74/ 75/ 76/ 77/ 78/ 79/ 80/ 81/ 82/ 83/ 84/ 85/ 86/ 87/ 88/ 89/ 90/ 91/ 92/ 93/ 94/ 95/ 96/ 97/ 98/ 99/ 100/ 101/ 102/ 103/ 104/ 105/ 106/ 107/ 108/ 109/ 110/ 111/ 112/ 113/ 114/ 115/ 116/ 117/ 118/ 119/ 120/ 121/ 122/ 123/ 124/ 125/ 126/ 127/ 128/ 129/ 130/ 131/ 132/ 133/ 134/ 135/ 136/ 137/ 138/ 139/ 140/ 141/ 142/ 143/ 144/ 145/ 146/ 147/ 148/ 149/ 150/ 151/ 152/ 153/ 154/ 155/ 156/ 157/ 158/ 159/ 160/ 161/ 162/ 163/ 164/ 165/ 166/ 167/ 168/ 169/ 170/ 171/ 172/ 173/ 174/ 175/ 176/ 177/ 178/ 179/ 180/ 181/ 182/ 183/ 184/ 185/ 186/ 187/ 188/ 189/ 190/ 191/ 192/ 193/ 194/ 195/ 196/ 197/ 198/ 199/ 200/ 201/ 202/ 203/ 204/ 205/ 206/ 207/ 208/ 209/ 210/ 211/ 212/ 213/ 214/ 215/ 216/ 217/ 218/ 219/ 220/ 221/ 

In [16]:
# 포켓몬 배틀의 결과를 저장합니다.

table = pd.concat([pokemon.loc[:,"name"], pd.DataFrame(result)], axis =1)

In [24]:
# columns의 이름을 지정해 누구와 싸웠는지 알게 합니다.

table.columns = ["name", ] + list(pokemon.loc[:,"name"])

In [71]:
# 확률의 평균과 확률에 따른 갯수를 구합니다.
# 예를 들어 MeanOf_p70 은 70% 이상 이기는 상대를 대상으로 한 평균을 구합니다.
# CountOf_p70 은 70% 이상 이기는 상대의 수를 구합니다.

for i in range(10):
    table["MeanOf_p" + str(i) + "0"] = table.iloc[:, 1:802][table.iloc[:, 1:802] >= 0.1*i].mean(axis = 1)

for i in range(10):
    table["CountOf_p" + str(i) + "0"] = table.iloc[:, 1:802][table.iloc[:, 1:802] >= 0.1*i].count(axis = 1)

In [101]:
# CountOf_p90을 기준으로 한 순위를 구합니다. 
# 해당 과정에서 ascending = Fasle 조건을 써주지 않아 Rank가 반대로 되어 있습니다. 이를 다음 과정에서 수정합니다.

table["TotalRank"] = table.loc[:, "CountOf_p90"].rank()
pokemon["TotalRank"] = table.loc[:, "CountOf_p90"].rank()

In [125]:
# 포켓몬 타입 내에서의 CountOf_p90을 기준으로 한 순위를 구합니다.
# 이 과정 역시 순서가 잘못되어 다음 과정에서 수정합니다.

types = pokemon.loc[:, "type1"].unique()
pokemon["TypeRank"] = np.nan
table["TypeRank"] = np.nan

for t in types:
    typerank = pokemon[pokemon["type1"] == t]["TotalRank"].rank()
    pokemon["TypeRank"].iloc[typerank.keys()] = typerank.values
    table["TypeRank"].iloc[typerank.keys()] = typerank.values

C:\Users\pqrs\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [126]:
# 정보를 저장합니다.

table.to_csv("pokemon_winner_rate.csv", encoding = "utf-8", index = False)
pokemon.to_csv("pokemon_stat_rank.csv", encoding = "utf-8", index = False)